In [ ]:
import cartopy.crs as ccrs
from cmocean import cm 
from dino import Experiment
from matplotlib import colors
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import cftime as cft
import xnemogcm as xn
import xgcm

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
dino_exp = Experiment(path, 'DensityRestoring/restart25')

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(12,6))
dino_exp.get_ACC().plot(color='black')
axs.set_ylabel('ACC transport [Sv]')
plt.title('')
plt.xlabel('time [years]')
plt.legend()
plt.grid()

In [ ]:
T_star      = dino_exp.get_T_star()
S_star      = dino_exp.get_S_star()
rho_star    = dino_exp.get_rho_star()

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,8), sharey=True)
T_star.isel(x_c=10, t_m=-1, y_c=slice(1,-1)).plot(color='midnightblue', y='gphit', ax=axs[0], label='December')
T_star.isel(x_c=10, t_m=-7, y_c=slice(1,-1)).plot(color='darkred', y='gphit', ax=axs[0], label='June')
S_star.isel(x_c=10, t_m=-1).plot(color='black', y='gphit', ax=axs[1])
rho_star.isel(x_c=10, t_m=-1, y_c=slice(1,-1)).plot(color='midnightblue', y='gphit', ax=axs[2], label='December')
rho_star.isel(x_c=10, t_m=-7, y_c=slice(1,-1)).plot(color='darkred', y='gphit', ax=axs[2], label='June')

axs[0].fill_betweenx(dino_exp.domain.gphiu.values[:,0], -2, 32, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[0].margins(x=0, y=0)
axs[0].set_xlabel(r"$T^{*}$ [°C]")
axs[0].set_title('')
axs[0].grid()
axs[0].legend()
#
axs[1].fill_betweenx(dino_exp.domain.gphiu.values[:,0], 34., 36.5, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[1].margins(x=0, y=0)
axs[1].set_xlabel(r"$S^{*}$ [psu]")
axs[1].set_title('')
axs[1].grid()
#
axs[2].fill_betweenx(dino_exp.domain.gphiu.values[:,0], 1022, 1028, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[2].margins(x=0, y=0)
axs[2].set_xlabel(r"$\rho^{*}$ [kg / m3]")
axs[2].set_title('')
axs[2].grid()
axs[2].legend()

In [ ]:
moc =  dino_exp.get_MOC(dino_exp.data.voce + dino_exp.data.voce_eiv)

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = (-moc).plot.contourf(x='y_f', y='rhop', cmap='RdBu_r', levels=36, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = dino_exp.data.rhop.isel(t_y=-1, x_c=30, y_c=slice(1,-1), z_c=slice(0,-1)).plot.contourf(x='gphit', y='gdept_0', cmap=cm.dense_r, levels=36, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\rho$ [kg/m^3]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('')

## Vertical mixing profile

In [ ]:
dino_exp_1y = Experiment(path, 'Get_a_vt')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,6), sharex=True, sharey=True)
dino_exp_1y.data.avt_no_evd.isel(x_c=30, y_c=30, t_d=-180, z_f=slice(1,-1)).plot(y='gdepw_0', yincrease=False, xscale='log', ax=axs[0], color='darkred')
dino_exp_1y.data.avt_no_evd.isel(x_c=30, y_c=100, t_d=-1, z_f=slice(1,-1)).plot(y='gdepw_0', yincrease=False, xscale='log', ax=axs[1], color='darkred')
axs[0].hlines(dino_exp_1y.data.mldr10_1max.isel(x_c=30, y_c=30, t_d=-180), 1e-6, 1, color='black', linestyles='dashed')
axs[1].hlines(dino_exp_1y.data.mldr10_1max.isel(x_c=30, y_c=100, t_d=-180), 1e-6, 1, color='black', linestyles='dashed')
plt.xlim(1e-6,1)
axs[0].set_title('Channel')
axs[1].set_title('Equator')
axs[0].set_xlabel('vertical heat diffusion without evd component [m2/s]')
axs[1].set_xlabel('vertical heat diffusion without evd component [m2/s]')
axs[1].set_ylabel(' ')

## Zonal averages of sst/sss/ssd

In [ ]:
#dino_exp.data.sst.isel(t_m=slice(-12,-1), y_c=slice(1,-1), x_c=slice(1,-1)).mean('x_c').plot.line(x='y_c')
(dino_exp.data.sst.isel(t_m=slice(-12,-1), y_c=slice(1,-1), x_c=slice(1,-1)) - T_star.isel(t_m=slice(-12,-1), y_c=slice(1,-1), x_c=slice(1,-1))).mean('x_c').plot.line(x='y_c')

In [ ]:
(dino_exp.data.sss.isel(t_m=-1, y_c=slice(1,-1), x_c=slice(1,-1))).mean('x_c').plot()

In [ ]:
nml = dino_exp.namelist['nameos']
ssd = (
    - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( dino_exp.data.sst - 10.)) * ( dino_exp.data.sst - 10.)
    + nml['rn_b0'] * (1. - 0.5 * nml['rn_lambda2'] * ( dino_exp.data.sss - 35.)) * ( dino_exp.data.sss - 35.)
    - nml['rn_nu'] * ( dino_exp.data.sst - 10.) * ( dino_exp.data.sss - 35.)
) + 1026

In [ ]:
(rho_star - ssd).isel(t_m=slice(-12,-1), y_c=slice(1,-1), x_c=slice(1,-1)).mean('x_c').plot.line(x='y_c')

## Zonal average of density

In [ ]:
vol_rho = dino_exp.transform_to_density(var=(dino_exp.domain.e1t * dino_exp.domain.e2t * dino_exp.data.e3t).isel(z_c=slice(0,-1), t_y=-1))

In [ ]:
vol_rho.sum(['y_c', 'x_c', 'rhop']).values

In [ ]:
(dino_exp.data.e3w * dino_exp.domain.e2t * dino_exp.domain.e1t).isel(t_y=-1).sum(['y_c', 'x_c', 'z_f']).values

In [ ]:
e3t_rho.mean('x_c').plot.contourf(x='y_c', y='rhop', cmap=cm.dense_r, levels=36)

In [ ]:
dino_exp.data.rhop.isel(t_y=-1, z_c=0, x_c=30, y_c=slice(1,-1)).min().values#.plot(x='gphit')

In [ ]:
a.values

In [ ]:
dino_exp.domain.gdept_1d.values

In [ ]:
dino_exp.data.sst.isel(t_m=-1).plot()

In [ ]:
dino_exp.domain.gdept_1d.isel(z_c=32).values

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = dino_exp.data.rhop.isel(t_y=-1, x_c=15, y_c=slice(1,-1), z_c=slice(0,-1)).plot.contourf(x='gphit', y='gdept_0', cmap=cm.dense_r, levels=36, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\rho$ [kg/m^3]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = dino_exp.data.toce.isel(t_y=-1, x_c=30, y_c=slice(1,-1), z_c=slice(0,-1)).plot.contourf(x='gphit', y='gdept_0', levels=36, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\rho$ [kg/m^3]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('temprature')

In [ ]:
from matplotlib.colors import LogNorm

soce = (dino_exp.data.isel(t_y=-1).where(dino_exp.domain.tmask == 1.)).soce.values.ravel()
toce = (dino_exp.data.isel(t_y=-1).where(dino_exp.domain.tmask == 1.)).toce.values.ravel()
vol  = (dino_exp.domain.e1t * dino_exp.domain.e2t * dino_exp.data.isel(t_y=-1).e3t).values.ravel()

smin, smax = 34, 36.5
tmin, tmax = -3, 28

s = np.linspace(smin, smax, 100)
t = np.linspace(tmin, tmax, 100)

Sg, Tg = np.meshgrid(s,t)

nml = dino_exp.namelist['nameos']

rho = (
    - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( Tg - 10.)) * ( Tg - 10.) 
    + nml['rn_b0'] * (1. - 0.5 * nml['rn_lambda2'] * ( Sg - 35.)) * ( Sg - 35.) 
    - nml['rn_nu'] * ( t - 10.) * ( Sg - 35.)
) + 1026

fig, ax = plt.subplots()
hb = ax.hexbin(soce, toce,
           C=vol, reduce_C_function=np.sum,
           extent=(34,36.5,-3,28), gridsize=50, bins='log',
           cmap=cm.matter)
plt.colorbar(hb)
cp = ax.contour(Sg,Tg, rho, levels=np.arange(1021, 1029, 0.5), linestyles='dashed', colors='black')
cl=plt.clabel(cp,fontsize=10,inline=True,fmt="%.1f")
ax.set_ylabel(r'Temperature ($^\circ$C)')
ax.set_xlabel("Salinity (g / kg)")

In [ ]:
plt.plot(vol,rho)

In [ ]:
mld_sep = dino_exp.data.mldr10_1.where(((dino_exp.data['t_m.year'] >= 1030) & (dino_exp.data['t_m.month'] == 9)), drop=True).mean('t_m')
mld_mar = dino_exp.data.mldr10_1.where(((dino_exp.data['t_m.year'] >= 1030) & (dino_exp.data['t_m.month'] == 3)), drop=True).mean('t_m')

mld_sep = mld_sep.assign_coords({'x_globe': mld_sep.glamt - 30})
mld_mar = mld_mar.assign_coords({'x_globe': mld_mar.glamt - 30})

In [ ]:
plt.figure(figsize=(10,10))
a = mld_sep.plot.contourf(
    y='gphit',
    x='x_globe',
    cmap=cm.deep_r,
    levels=30,
    subplot_kws=dict(projection=ccrs.Robinson()),

    transform=ccrs.PlateCarree(),
    #add_colorbar=False,
)
a.axes.gridlines(
    draw_labels=["x", "y", "geo"],
    ylocs=[-70, -45, -20, 0, 20, 45, 70],
    xlocs=[0]
)
plt.title('')
plt.grid('m')
plt.tight_layout()

In [ ]:
(86400 * dino_exp.data.saltflx / dino_exp.data.sss).isel(t_m=slice(-13,-1)).mean('t_m').plot()

In [ ]:
dino_exp.data.saltflx.where(dino_exp.domain.tmask.isel(z_c=0)==1.0).isel(t_m=slice(-13,-1)).mean('t_m').plot()

In [ ]:
def scurve(x, x0, dx):
    """Returns 0 for x<x0 or x>x+dx, and a cubic in between."""
    s = np.minimum(1, np.maximum(0, (x-x0)/dx))
    return (3 - 2*s)*( s*s )

def itau(ytau,taud,ys):
    """Returns a profile tau(ys) that uses s-curves between node,valuesa ytau,taud"""
    taus = 0.*ys
    ks = 0
    for i in range(len(ys)):
        y = ys[i]
        if y>=ytau[ks+1]:
            ks=min(len(ytau)-2,ks+1)
        taus[i] = taud[ks] + ( taud[ks+1] - taud[ks]) * scurve(y, ytau[ks], ytau[ks+1]-ytau[ks])
    return taus

def S_star(rho, T):
    nml = dino_exp.namelist['nameos']
    s_star = ( rho - 1026.
    + nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( T - 10.)) * ( T - 10.)
    ) / nml['rn_b0'] + 35.
    return(s_star)

def Rho_star(toce, soce):
    nml = dino_exp.namelist['nameos']
    rho_star = (
        - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( toce - 10.) ) * ( toce - 10.) 
        + nml['rn_b0'] * ( soce - 35.) 
    ) + 1026
    return(rho_star)

ysteps = np.linspace(-70,70,100)

In [ ]:
def S_of_phi(experiment):
    S_s     = 35.
    S_n     = 35.1
    S_eq    = 37.25
    S_b     = (experiment.domain.gphit <= 0.) * S_s + (experiment.domain.gphit > 0.) * S_n
    S       = S_b + (S_eq - S_b) * 0.5 * (1. + np.cos(np.pi * experiment.domain.gphit / 70.)) - 1.25 * np.exp(- (experiment.domain.gphit / 7.5)**2)
    return(S)

def T_of_phi(experiment):
    T_s     = 0.
    T_n     = 2.
    T_eq    = 27.
    T_b     = (experiment.domain.gphit <= 0.) * T_s + (experiment.domain.gphit > 0.) * T_n
    T       = T_b + (T_eq - T_b) * np.sin(np.pi * (experiment.domain.gphit + 70.) / 140)
    return(T)


In [ ]:
S_of_phi(dino_exp).isel(x_c=30).plot(figsize=(20,5), x='gphit')
plt.hlines(38., -70, 70)

In [ ]:
T_of_phi(dino_exp).isel(x_c=30).plot(figsize=(20,5), x='gphit')

In [ ]:
Rho_star(T_of_phi(dino_exp), S_of_phi(dino_exp)).isel(x_c=30).plot(figsize=(20,5), x='gphit')

In [ ]:
(Rho_star(T_of_phi(dino_exp), S_of_phi(dino_exp)).isel(x_c=30, y_c=0) - Rho_star(T_of_phi(dino_exp), S_of_phi(dino_exp)).isel(x_c=30, y_c=-1)).values

In [ ]:
dino_exp.data.sst.isel(x_c=30, t_m=-1).plot(x='gphit', label='Dezember')
dino_exp.data.sst.isel(x_c=30, t_m=-4).plot(x='gphit', label='September')
dino_exp.data.sst.isel(x_c=30, t_m=-7).plot(x='gphit', label='Juni')
dino_exp.data.sst.isel(x_c=30, t_m=-10).plot(x='gphit', label='Maerz')
plt.vlines([-10,10], -3, 30)
plt.legend()

In [ ]:
dino_exp.get_T_star().isel(x_c=30, t_m=-1).plot(x='gphit', label='Dezember')
dino_exp.get_T_star().isel(x_c=30, t_m=-4).plot(x='gphit', label='September')
dino_exp.get_T_star().isel(x_c=30, t_m=-7).plot(x='gphit', label='Juni')
dino_exp.get_T_star().isel(x_c=30, t_m=-6).plot(x='gphit', label='Juli')
dino_exp.get_T_star().isel(x_c=30, t_m=-10).plot(x='gphit', label='Maerz')
plt.legend()

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
testF = Experiment(path, 'testForcing')

In [ ]:
sstar = testF.get_S_star()
tstar = testF.get_T_star()
rhostar = testF.get_rho_star()

In [ ]:
rhostar.isel(t_0=6, x_c=30).plot(x='gphit')

In [ ]:
(rhostar.isel(x_c=30, y_c=1) - rhostar.isel(x_c=30, y_c=-2)).plot(x='t_0')

In [ ]:
testF.data.qns.isel(x_c=30, y_c=-1).idxmax().values

In [ ]:
dino_exp.data.qns.isel(x_c=30, t_m=slice(-13,-1)).plot()

In [ ]:
dino_exp.domain.gphit.isel(x_c=30).values